In [137]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 12)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import ipympl
%matplotlib widget

# data read

In [3]:
# matplot setting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/NGULIM.ttf").get_name()
font_name = font_manager.FontProperties(
    fname="/usr/share/fonts/truetype/nanum/NanumGothic.ttf").get_name()
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False
# plt.style.use('dark_background')

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df_final = "../../data/가공데이터/df_final.csv"
df_final = pd.read_csv(df_final)
df_final['날짜'] = pd.to_datetime(df_final['날짜'], format='%Y-%m-%d')
df_final['week'] = df_final['날짜'].dt.week
df_final['month'] = df_final['날짜'].dt.month
df_final.head()

,Unnamed: 0,날짜,구_x,동,예보,pm10,co2,vocs,noise,temp,...,임신/육아_비중,업종코드,성별코드,나이코드,이용건수,이용금액,구_y,FLOW_POP_CNT,week,month
0,0,2018-04-01,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,...,0.00925,10,F,25,39,4936,종로구,6892.47,13,4
1,1,2018-04-01,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,...,0.00925,21,F,25,18,565,종로구,6892.47,13,4
2,2,2018-04-01,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,...,0.00925,22,F,25,149,1709,종로구,6892.47,13,4
3,3,2018-04-01,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,...,0.00925,34,F,25,13,101,종로구,6892.47,13,4
4,4,2018-04-01,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,...,0.00925,40,F,25,556,4611,종로구,6892.47,13,4


In [6]:
del df_final['Unnamed: 0']
del df_final['구_y']
df_final = df_final.drop(
    columns=[
        'pm25',
        'co2',
        'vocs',
        'noise',
        'temp',
        'humi']).dropna()

In [7]:
df_final.성별코드 = df_final.성별코드.replace('F', -1)
df_final.성별코드 = df_final.성별코드.replace('M', 1)

In [8]:
df_final.rename(columns={'구_x': '구', 'FLOW_POP_CNT': '유동인구'}, inplace=True)

In [197]:
map_card_code = {10: '숙박',
 20: '레저용품',
 21: '레저업소',
 22: '문화취미',
 30: '가구',
 31: '전기',
 32: '주방용구',
 33: '연료판매',
 34: '광학제품',
 35: '가전',
 40: '유통업',
 42: '의복',
 43: '작물',
 44: '신변잡화',
 50: '서적문구',
 52: '사무통신',
 60: '자동차판매',
 62: '자동차정비',
 70: '의료기관',
 71: '보건위생',
 80: '요식업소',
 81: '음료식품',
 92: '수리서비스'}
df_final['업종코드'] = pd.DataFrame(df_final['업종코드']).applymap(map_card_code.get)

In [9]:
# null check -> None
df_final.isna().sum()

날짜            0
구             0
동             0
예보            0
pm10          0
매출지수          0
식사_비중         0
간식_비중         0
마실거리_비중       0
홈&리빙_비중       0
헬스&뷰티_비중      0
취미&여가활동_비중    0
사회활동_비중       0
임신/육아_비중      0
업종코드          0
성별코드          0
나이코드          0
이용건수          0
이용금액          0
유동인구          0
week          0
month         0
dtype: int64

# data 분석

## 미세먼지 vs 소비(card & gs)

### 미세먼지 vs gs

In [201]:
# 일정 기간내 correlation map (pm10 vs card data)
import seaborn as sns
f, ax = plt.subplots(figsize=(10, 8))

state1 = df_final.month >= 1 
state2 = df_final.month <= 3 

df_pm10_gs = df_final[state1 & state2]
corr_pm10_gs = df_pm10_gs[['pm10',
                           '매출지수',
                           '식사_비중',
                           '간식_비중',
                           '마실거리_비중',
                           '홈&리빙_비중',
                           '헬스&뷰티_비중',
                           '취미&여가활동_비중',
                           '사회활동_비중',
                           '임신/육아_비중']].corr()
sns.heatmap(
    corr_pm10_gs,
    mask=np.zeros_like(
        corr_pm10_gs,
        dtype=np.bool),
    cmap=sns.diverging_palette(
            220,
            10,
            as_cmap=True),
    square=True,
    ax=ax,
    vmin=-1,
    vmax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# 임신육아, 헬스뷰티 0.15이상
corr_pm10_gs['pm10']

pm10          1.000000
매출지수          0.034321
식사_비중        -0.020731
간식_비중        -0.047065
마실거리_비중      -0.006591
홈&리빙_비중      -0.040734
헬스&뷰티_비중      0.146758
취미&여가활동_비중   -0.047563
사회활동_비중       0.052782
임신/육아_비중      0.154219
Name: pm10, dtype: float64

## 미세먼지 vs 성연령별 flow

### year, 미세먼지 vs 성연령별 flow

In [21]:
# 임시 func
def mise_corr(index, df_final):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '유동인구', index]]
    df = df.drop_duplicates(keep='first')
    df = df.drop(
        ['날짜', '구', '동'],
        axis=1).groupby(
        ['성별코드', '나이코드']).corr()['유동인구'].unstack(
        level=2).drop(
        columns='유동인구')

    return df

corr_mise_sa = mise_corr('pm10', df_final)
corr_mise_sa.style.bar(
    subset='pm10',
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

### week, 미세먼지 vs 성연령별 flow

In [49]:
def mise_week_corr(index, week, df_final):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '유동인구', 'pm10']]
    state1 = df_final.week  == week
    df = df[state1] 
    df = df.drop_duplicates(keep='first')
    df = df.drop( ['날짜', '구', '동'], axis=1).groupby( ['성별코드', '나이코드']).corr()['유동인구'].unstack( level=2).drop( columns='유동인구')
    df = df.rename(columns={'pm10':week})

    return df
week_all = df_final.week.unique()
week_all.sort()
corr_mise_sa_week = (mise_week_corr('pm10', week, df_final) for week in week_all)
corr_mise_sa_week_all = pd.concat(corr_mise_sa_week, axis=1)
corr_mise_sa_week_all.style.bar(
    subset=week_all,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

### month, 미세먼지 vs 성연령별 flow

In [57]:
def mise_month_corr(index, month, df_final):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '유동인구', 'pm10']]
    state1 = df_final.month  == month
    df = df[state1] 
    df = df.drop_duplicates(keep='first')
    df = df.drop( ['날짜', '구', '동'], axis=1).groupby( ['성별코드', '나이코드']).corr()['유동인구'].unstack( level=2).drop( columns='유동인구')
    df = df.rename(columns={'pm10':month})

    return df
month_all = df_final.month.unique()
month_all.sort()
corr_mise_sa_month = (mise_month_corr('pm10', month, df_final) for month in month_all)
corr_mise_sa_month_all = pd.concat(corr_mise_sa_month, axis=1)
corr_mise_sa_month_all.style.bar(
    subset=month_all,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

### 좋음 나쁨정도(일별)를 통한 미세먼지 vs 성연령별 flow

In [11]:
def mise_fcast_corr(index, fcast, df_final):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '유동인구', 'pm10']]
    state1 = df_final.예보  == fcast
    df = df[state1] 
    df = df.drop_duplicates(keep='first')
    df = df.drop( ['날짜', '구', '동'], axis=1).groupby( ['성별코드', '나이코드']).corr()['유동인구'].unstack( level=2).drop( columns='유동인구')
    df = df.rename(columns={'pm10':fcast})

    return df
fcast_all = df_final.예보.unique()
fcast_all.sort()
corr_mise_sa_fcast = (mise_fcast_corr('pm10', fcast, df_final) for fcast in fcast_all)
corr_mise_sa_fcast_all = pd.concat(corr_mise_sa_fcast, axis=1)
corr_mise_sa_fcast_all.style.bar(
    subset=fcast_all,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

### 미세먼지 vs 성연령별 flow (기간 별)

In [138]:
df_final.groupby(['month']).pm10.agg({'pm10':['mean','min','max']})

pm10                       
            mean        min         max
month                                  
1      87.796104  26.950000  399.573611
2      83.716824  14.725000  237.950694
3      87.077800  12.754861  435.559028
4      51.752761   1.779167  138.093750
5      48.012935   5.095833  130.948611
6      42.129153   2.921528   93.612500
7      33.718234   2.068750   88.917361
8      35.810453   2.749306   97.299306
9      31.762210   0.878472   70.368056
10     40.317754   1.449306  148.833333
11     61.846607   6.040278  245.084028
12     58.661901  16.747222  130.121528

In [179]:
df_yebo_ratio = df_final.groupby(['month']).예보.value_counts() / df_final.groupby(['month']).예보.count()

In [180]:
df_yebo_ratio

month  예보  
1      보통      0.485417
       나쁨      0.427874
       매우나쁨    0.076762
       좋음      0.009947
2      보통      0.499883
                 ...   
11     좋음      0.133464
       매우나쁨    0.017996
12     보통      0.702555
       나쁨      0.202424
       좋음      0.095021
Name: 예보, Length: 39, dtype: float64

In [182]:
# 나쁨 & 매우나쁨 비율 합
df_yebo_ratio.unstack(level=0).T[['나쁨', '매우나쁨']].fillna(0).sum(axis=1)

month
1     0.504636
2     0.486936
3     0.344310
4     0.123115
5     0.157423
6     0.017305
7     0.002054
8     0.007532
9     0.000000
10    0.035330
11    0.226518
12    0.202424
dtype: float64

#### 1,2,3 월

In [185]:
def mise_corr(index, df_final):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '유동인구', index]]
    df = df.drop_duplicates(keep='first')
    df = df.drop(
        ['날짜', '구', '동'],
        axis=1).groupby(
        ['성별코드', '나이코드']).corr()['유동인구'].unstack(
        level=2).drop(
        columns='유동인구')

    return df

In [185]:
state1 = df_final.month >= 1 
state2 = df_final.month <= 3 
# state3 = df_final.예보 == '좋음'

corr_mise_sa = mise_corr('pm10', df_final[state1 & state2])
# corr_mise_sa = mise_corr('pm10', df_final[state1 | state2 & state3])
corr_mise_sa.style.bar(
    subset='pm10',
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 4~10 월

In [186]:
state1 = df_final.month >= 4 
state2 = df_final.month <= 10 
# state3 = df_final.예보 == '좋음'

corr_mise_sa = mise_corr('pm10', df_final[state1 & state2])
# corr_mise_sa = mise_corr('pm10', df_final[state1 | state2 & state3])
corr_mise_sa.style.bar(
    subset='pm10',
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 11~12 월

In [187]:
state1 = df_final.month >= 11
state2 = df_final.month <= 12 
# state3 = df_final.예보 == '좋음'

corr_mise_sa = mise_corr('pm10', df_final[state1 & state2])
# corr_mise_sa = mise_corr('pm10', df_final[state1 | state2 & state3])
corr_mise_sa.style.bar(
    subset='pm10',
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

## gs지수 vs 성연령별 유동인구 분포 확인

### 1~12월 gs지수 vs 성연령별 유동인구 분포 확인

In [12]:
def sa_corr(index, df_final):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '유동인구', index]]
    df = df.drop_duplicates(keep='first')
    df = df.drop(
        ['날짜', '구', '동'],
        axis=1).groupby(
        ['성별코드', '나이코드']).corr()['유동인구'].unstack(
        level=2).drop(
        columns='유동인구')

    return df


# df_final_ is scaled
df_final_ = df_final.copy()
gs_index = ['식사_비중', '간식_비중', '마실거리_비중',
            '홈&리빙_비중', '헬스&뷰티_비중', '취미&여가활동_비중', '사회활동_비중', '임신/육아_비중']
for index in gs_index:
    df_final_[index] = df_final['매출지수'].mul(df_final[index])

gs_index.append('매출지수')

corr_gs_sa = (sa_corr(index, df_final_) for index in gs_index)
corr_gs_sa_all = pd.concat(corr_gs_sa, axis=1)
corr_gs_sa_all.style.bar(
    subset=gs_index,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 1~3월 gs지수 vs 성연령별 유동인구 분포 확인

In [189]:
# df_final_ is scaled
df_final_ = df_final.copy()
gs_index = ['식사_비중', '간식_비중', '마실거리_비중',
            '홈&리빙_비중', '헬스&뷰티_비중', '취미&여가활동_비중', '사회활동_비중', '임신/육아_비중']
for index in gs_index:
    df_final_[index] = df_final['매출지수'].mul(df_final[index])

gs_index.append('매출지수')

# 월별
state1 = df_final.month >= 1 
state2 = df_final.month <= 3 
corr_gs_sa = (sa_corr(index, df_final_[state1 & state2]) for index in gs_index)
corr_gs_sa_all = pd.concat(corr_gs_sa, axis=1)
corr_gs_sa_all.style.bar(
    subset=gs_index,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 4~10월 gs지수 vs 성연령별 유동인구 분포 확인

In [191]:
# df_final_ is scaled
df_final_ = df_final.copy()
gs_index = ['식사_비중', '간식_비중', '마실거리_비중',
            '홈&리빙_비중', '헬스&뷰티_비중', '취미&여가활동_비중', '사회활동_비중', '임신/육아_비중']
for index in gs_index:
    df_final_[index] = df_final['매출지수'].mul(df_final[index])

gs_index.append('매출지수')

# 월별
state1 = df_final.month >= 4 
state2 = df_final.month <= 10 
corr_gs_sa = (sa_corr(index, df_final_[state1 & state2]) for index in gs_index)
corr_gs_sa_all = pd.concat(corr_gs_sa, axis=1)
corr_gs_sa_all.style.bar(
    subset=gs_index,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 11~12월 gs지수 vs 성연령별 유동인구 분포 확인

In [192]:
# df_final_ is scaled
df_final_ = df_final.copy()
gs_index = ['식사_비중', '간식_비중', '마실거리_비중',
            '홈&리빙_비중', '헬스&뷰티_비중', '취미&여가활동_비중', '사회활동_비중', '임신/육아_비중']
for index in gs_index:
    df_final_[index] = df_final['매출지수'].mul(df_final[index])

gs_index.append('매출지수')

# 월별
state1 = df_final.month >= 11
state2 = df_final.month <= 12 
corr_gs_sa = (sa_corr(index, df_final_[state1 & state2]) for index in gs_index)
corr_gs_sa_all = pd.concat(corr_gs_sa, axis=1)
corr_gs_sa_all.style.bar(
    subset=gs_index,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

## card vs 성연령별 유동인구 분포 확인
* [성별코드, 나이코드, 업종코드]의 모든 cased에 대해서 이용금액이 존재하지 않아
* 각 case마다 이용금액을 고려
* (2 x 10 x 23) = 460
    * 매우많으므로 분석방향 고려해야함
    * 일정 corr 값을 넘는 것을 추려야함

### 이용금액 vs flow

In [194]:
# corr 이용금액
def card_corr(sex, age, b_code, df_final, corr_value):
    df = df_final[['날짜', '구', '동', '성별코드', '나이코드', '업종코드','유동인구', corr_value]]
    state1 = df_final.성별코드 == sex
    state2 = df_final.나이코드 == age
    state3 = df_final.업종코드 == b_code
    df = df[state1 & state2 & state3]
    df = df.drop(
        ['날짜', '구', '동'],
        axis=1).groupby(['성별코드', '나이코드', '업종코드']).corr()['유동인구'].unstack(level=3).drop(columns='유동인구').stack().unstack(level=2)

    return df

In [180]:
import itertools
sex_all = df_final.성별코드.unique()
age_all = df_final.나이코드.unique()
b_code_all = df_final.업종코드.unique()

corr_card_sa = (card_corr(sex, age, b_code, df_final, '이용금액') for sex, age, b_code in itertools.product(sex_all, age_all, b_code_all) )
corr_card_sa_all = pd.concat(corr_card_sa, axis=0)
corr_card_sa_all_fillna = corr_card_sa_all.sum(level=[0,1], skipna=True)
corr_card_sa_all_fillna.sort_index(inplace=True)

corr_card_sa_all_fillna.style.bar(
    subset=corr_card_sa_all_fillna.columns,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 1~3월 이용금액 vs flow

In [199]:
import itertools
sex_all = df_final.성별코드.unique()
age_all = df_final.나이코드.unique()
b_code_all = df_final.업종코드.unique()

state1 = df_final.month >= 1 
state2 = df_final.month <= 3 

corr_card_sa = (card_corr(sex, age, b_code, df_final[state1 & state2], '이용금액') for sex, age, b_code in itertools.product(sex_all, age_all, b_code_all) )
corr_card_sa_all = pd.concat(corr_card_sa, axis=0)
corr_card_sa_all_fillna = corr_card_sa_all.sum(level=[0,1], skipna=True)
corr_card_sa_all_fillna.sort_index(inplace=True)

corr_card_sa_all_fillna.style.bar(
    subset=corr_card_sa_all_fillna.columns,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 4~10월 이용금액 vs flow

In [200]:
import itertools
sex_all = df_final.성별코드.unique()
age_all = df_final.나이코드.unique()
b_code_all = df_final.업종코드.unique()

state1 = df_final.month >= 4 
state2 = df_final.month <= 10 

corr_card_sa = (card_corr(sex, age, b_code, df_final[state1 & state2], '이용금액') for sex, age, b_code in itertools.product(sex_all, age_all, b_code_all) )
corr_card_sa_all = pd.concat(corr_card_sa, axis=0)
corr_card_sa_all_fillna = corr_card_sa_all.sum(level=[0,1], skipna=True)
corr_card_sa_all_fillna.sort_index(inplace=True)

corr_card_sa_all_fillna.style.bar(
    subset=corr_card_sa_all_fillna.columns,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

#### 11~12월 이용금액 vs flow

In [198]:
import itertools
sex_all = df_final.성별코드.unique()
age_all = df_final.나이코드.unique()
b_code_all = df_final.업종코드.unique()

state1 = df_final.month >= 11 
state2 = df_final.month <= 12 

corr_card_sa = (card_corr(sex, age, b_code, df_final[state1 & state2], '이용금액') for sex, age, b_code in itertools.product(sex_all, age_all, b_code_all) )
corr_card_sa_all = pd.concat(corr_card_sa, axis=0)
corr_card_sa_all_fillna = corr_card_sa_all.sum(level=[0,1], skipna=True)
corr_card_sa_all_fillna.sort_index(inplace=True)

corr_card_sa_all_fillna.style.bar(
    subset=corr_card_sa_all_fillna.columns,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

### 이용건수(sum)

In [190]:
# 이용건수 확인
df_card_num = df_final[['성별코드', '나이코드', '업종코드', '이용건수']]
df_card_num = df_card_num.groupby(['성별코드', '나이코드', '업종코드']).sum().unstack(level=2)
df_card_num.style.bar(
    subset=df_card_num.columns,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=df_card_num.min().min(),
    vmax=df_card_num.max().max())

### 이용건수(sum) vs flow

In [192]:
import itertools
sex_all = df_final.성별코드.unique()
age_all = df_final.나이코드.unique()
b_code_all = df_final.업종코드.unique()

corr_card_sa_num = (card_corr(sex, age, b_code, df_final, '이용건수') for sex, age, b_code in itertools.product(sex_all, age_all, b_code_all) )
corr_card_sa_num_all = pd.concat(corr_card_sa_num, axis=0)
corr_card_sa_num_all_fillna = corr_card_sa_num_all.sum(level=[0,1], skipna=True)
corr_card_sa_num_all_fillna.sort_index(inplace=True)

corr_card_sa_num_all_fillna.style.bar(
    subset=corr_card_sa_num_all_fillna.columns,
    align='mid',
    color=[
        '#d65f5f',
        '#5fba7d'],
    vmin=-1,
    vmax=1)

# profiling

In [40]:
import pandas_profiling
pandas_profiling.ProfileReport(
                               df_final
                               [['pm10', '성별코드', '나이코드', '이용건수', '이용금액',
                                 'FLOW_POP_CNT']].sample(n=10000))

KeyError: "['FLOW_POP_CNT'] not in index"

# TEST CODE

In [24]:
df_final[['예보','pm10']].groupby('예보').agg(['min', 'max'])

pm10            
             min         max
예보                          
나쁨     80.019526  149.904861
매우나쁨  150.061111  435.559028
보통     30.006250   79.972917
좋음      0.878472   29.990972

In [67]:
# 월별 미세먼지 나쁨이상 빈도수
df_final.groupby(['날짜']).pm10.agg(['mean', 'min', 'max'])

,mean,min,max
날짜,,,
2018-04-01,64.414286,45.384240,90.150694
2018-04-02,66.898656,50.623431,90.823611
2018-04-03,65.482506,45.456583,81.932639
2018-04-04,15.085985,4.499306,43.870175
2018-04-05,8.981919,2.903472,15.961111
...,...,...,...
2019-03-27,95.095784,77.293056,201.000000
2019-03-28,82.188492,59.938889,123.572222
2019-03-29,40.539337,32.267361,62.290278


In [120]:
df_final.groupby(['날짜']).pm10.describe()

,count,mean,std,min,25%,50%,75%,max
날짜,,,,,,,,
2018-04-01,5122.0,64.414286,10.378281,45.384240,55.554167,63.603800,71.583333,90.150694
2018-04-02,5419.0,66.898656,9.497265,50.623431,60.178977,65.076389,74.570432,90.823611
2018-04-03,5089.0,65.482506,10.071404,45.456583,57.281250,62.797917,72.707810,81.932639
2018-04-04,5009.0,15.085985,8.960627,4.499306,10.243750,13.441667,16.062500,43.870175
2018-04-05,4305.0,8.981919,3.563445,2.903472,5.479167,9.480309,11.940972,15.961111
...,...,...,...,...,...,...,...,...
2019-03-27,4209.0,95.095784,24.054407,77.293056,84.268750,88.293056,95.149306,201.000000
2019-03-28,4144.0,82.188492,11.550581,59.938889,76.332639,81.944444,87.837500,123.572222
2019-03-29,4257.0,40.539337,6.209311,32.267361,36.893750,39.257639,42.961806,62.290278


In [123]:
df_std_ratio = df_final.groupby(['날짜']).pm10.describe()['std'] / df_final.groupby(['날짜']).pm10.describe()['mean']

In [135]:
df_std_ratio[df_std_ratio > 0.5]

날짜
2018-04-04    0.593970
2018-10-06    0.685287
2019-01-24    0.539102
2019-01-26    0.569738
2019-02-08    0.554256
2019-02-09    0.535822
2019-02-10    0.560738
2019-02-11    0.506925
2019-02-13    0.540855
dtype: float64